In [2]:
## Multi- Spectrum Fitting by Segment

import numpy as np
import numpy.ma as ma
import pandas as pd
import os
from scipy import fft, arange, signal
from scipy.interpolate import interp1d
from scipy.optimize import fmin, fmin_powell
from lmfit import minimize, Parameters, report_fit, Model, Minimizer
from lmfit.models import GaussianModel, LinearModel, Model, VoigtModel
from scipy.stats import linregress
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import interpolate

import seaborn as sns
import pybroom as br

import sys
sys.path.append(r'C:\Users\ema3\OneDrive - National Institute of Standards and Technology (NIST)\Python Scripts\HAPI')# set location of HAPI.py module
sys.path.append(r'C:\Users\ema3\OneDrive - National Institute of Standards and Technology (NIST)\Projects\Cold Cavity - O2 A Band')# set location of HAPI.py module
%matplotlib inline
from hapi import *
from hapi_supplemental_def import *
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("poster")




In [5]:
path = r'C:\Users\ema3\Desktop\Cold Cavity Oxygen A Band Pre-Processed\0.2%\237K\Etalon - Fitted\Seg2\Segment-wise Etalon Fit'# Location of the Summary Data File'
os.chdir(path)

save_name = 'Test'
filelist = ['28Feb18_237.5K_0.2%_50Torr_etalon_Seg2_etalon', '27Feb18_237.5K_0.2%_100Torr_etalon_Seg2_etalon', '26Feb18_237.5K_0.2%_200Torr_etalon_Seg2_etalon', 
           '23Feb18_237.5K_0.2%_400Torr_etalon_Seg2_etalon', '22Feb18_237.5K_0.2%_700Torr_etalon_Seg2_etalon', '21Feb18_237.5K_0.2%_1000Torr_etalon_Seg2_etalon']
pressure_list = [50, 100, 200, 400, 700, 1000] #, 700, 1000

temperature = 237
sample = 1        

if sample == 1:
    percent_oxygen = 0.2022 # percentage
    isotopes = [36]
    natural_abundance = False
elif sample == 2:
    percent_oxygen = 1.9734 
    isotopes = [36, 37, 38]
    natural_abundance = True
elif sample == 3:
    percent_oxygen = 18.069
    isotopes = [36]
    natural_abundance = False

if temperature == 290:
    etalon_mirror = [0.00598, 24.648458*10**9 / 29979245800] 
    etalon_window = [0.00108,7.352352*10**9 / 29979245800]
    etalon_slow = [0.000884, 158.269045*10**9 / 29979245800]
    etalon_fast  = [0.000437, 12.349535*10**9 / 29979245800]
elif temperature == 273:
    if sample != 1:
        etalon_mirror = [0.00566, 24.680022*10**9 / 29979245800] 
        etalon_window = [0.00108,7.352778*10**9 / 29979245800]
        etalon_slow = [0.000813, 154.408341*10**9 / 29979245800]
        etalon_fast  = [0.000440, 12.36535*10**9 / 29979245800]
    elif sample == 1:
        etalon_mirror = [0.00580, 24.780043*10**9 / 29979245800] 
        etalon_window = [0.00109,7.356575*10**9 / 29979245800]
        etalon_slow = [0.00139, 156.940272*10**9 / 29979245800]
        etalon_fast  = [0.000432, 12.335917*10**9 / 29979245800]
elif temperature == 255:
    if sample != 1:
        etalon_mirror = [0.00507, 24.756192*10**9 / 29979245800] 
        etalon_window = [0.00108,7.354223*10**9 / 29979245800]
        etalon_slow = [0, 154.408*10**9 / 29979245800]
        etalon_fast  = [0.000429, 12.352679*10**9 / 29979245800]
    elif sample == 1:
        etalon_mirror = [0.00501, 24.824664*10**9 / 29979245800] 
        etalon_window = [0.00107,7.360413*10**9 / 29979245800]
        etalon_slow = [0.00143, 170.669565*10**9 / 29979245800]
        etalon_fast  = [0.000443, 12.356168*10**9 / 29979245800]
elif temperature == 237:
    etalon_mirror = [0.005000, 24.727861*10**9 / 29979245800] 
    etalon_window = [0.00107,7.354798*10**9 / 29979245800]
    etalon_slow = [0.000921, 171.68201*10**9 / 29979245800]
    etalon_fast  = [0.000441, 12.363931*10**9 / 29979245800]


DATA = pd.DataFrame() 
count = 0
for filename in filelist:
    temp = pd.read_csv(filename + ".csv")
    temp['Nominal Pressure'] = pressure_list[count]
    #temp['Frequency'] = temp['Total Frequency /MHz']
    #temp['FSR Residuals'] = temp['Total Frequency /MHz'] - temp['Frequency (FSR) /MHz']
    #temp.loc[(temp['FSR Residuals'] < -100)|(temp['FSR Residuals'] > 100) , 'Frequency'] = temp[(temp['FSR Residuals'] < -100)|(temp['FSR Residuals'] > 100)]['Frequency (FSR) /MHz']
    
    DATA = DATA.append(temp)
    count += 1

#DATA['Wavenumber'] = DATA['Frequency']*1e6 / 29979245800
DATA['Filter'] = len(DATA)*[0]
DATA['Residuals'] = len(DATA)*np.nan
DATA['Etalons'] = len(DATA)*np.nan
DATA['Model'] = len(DATA)*np.nan

DATA = DATA[(DATA['Wavenumber (FSR)'] > 13080)]

DATA = DATA.reset_index(drop=True)

hapi = r'C:\Users\ema3\OneDrive - National Institute of Standards and Technology (NIST)\Projects\Cold Cavity - O2 A Band' # Location of the Summary Data File
os.chdir(hapi)
db_begin()


Using .

Drouin_2016
                     Lines parsed: 489
HITRAN
                     Lines parsed: 489
HITRAN_CO2
                     Lines parsed: 1712
HITRAN_O2
                     Lines parsed: 196
Long_2011
                     Lines parsed: 559
sampletab
                     Lines parsed: 3
SDVP check
                     Lines parsed: 2


In [6]:
db_begin()
fetch_by_ids('HITRAN_O2',[36,37,38],12820,13333,ParameterGroups=['sdngp', 'linemixing', 'voigt_he']) 

Using .

Drouin_2016
                     Lines parsed: 489
HITRAN
                     Lines parsed: 489
HITRAN_CO2
                     Lines parsed: 1712
HITRAN_O2
                     Lines parsed: 196
Long_2011
                     Lines parsed: 559
sampletab
                     Lines parsed: 3
SDVP check
                     Lines parsed: 2

Data is fetched from http://hitran.org

BEGIN DOWNLOAD: HITRAN_O2
  65536 bytes written to ./HITRAN_O2.data
  65536 bytes written to ./HITRAN_O2.data
Header written to ./HITRAN_O2.header
END DOWNLOAD
                     Lines parsed: 488
PROCESSED


In [7]:
IntensityThreshold = 1e-30
Fit_Intensity = 1e-25
wave_range = 1.5


def max_iter(pars, iter, resid, *args, **kws):
        if iter > 2500:
            return True
        else:
            return False
def etalon(x, amp, freq, phase):
    #period
    return amp*np.sin((2*np.pi * freq)*x+ phase)

def etalon_linear(x, amp, f0, phase, alpha):
    #based on a linear chirp
    f1 = alpha*f0
    k = (f1 - f0) / (x[-1] - x[0])
    return amp*np.sin(2*np.pi*(f0*x + 0.5*k*x**2)+ phase)

def etalon_model(params, x_,pressure_num ,DF):
    #Etalon Parameters
    ##Mirror Etalon
    amp_1 = np.float(params['amp_1_' + str(pressure_num)])
    freq_1 = np.float(params['freq_1_' + str(pressure_num)])
    phase_1 = np.float(params['phase_1_'+ str(pressure_num)])
    #alpha_1 = np.float(params['alpha_1_'+ str(pressure_num)])

    ## Other Etalon
    amp_2 = np.float(params['amp_2_'+ str(pressure_num)])
    freq_2 = np.float(params['freq_2_'+ str(pressure_num)])
    phase_2 = np.float(params['phase_2_'+ str(pressure_num)])
    #alpha_2 = np.float(params['alpha_2_'+ str(pressure_num)])

    ## Slow Other Etalon
    amp_3 = np.float(params['amp_3_'+ str(pressure_num)])
    freq_3= np.float(params['freq_3_'+ str(pressure_num)])
    phase_3 = np.float(params['phase_3_'+ str(pressure_num)])
    #alpha_3 = np.float(params['alpha_3_'+ str(pressure_num)])


    ## Fast Other Etalon
    amp_4 = np.float(params['amp_4_'+ str(pressure_num)])
    freq_4= np.float(params['freq_4_'+ str(pressure_num)])
    phase_4 = np.float(params['phase_4_'+ str(pressure_num)])
    #alpha_4 = np.float(params['alpha_4_'+ str(pressure_num)])



    
    etalon_1 = etalon(x_ , amp_1, freq_1, phase_1)
    etalon_2 = etalon(x_, amp_2, freq_2, phase_2)
    etalon_3 = etalon(x_, amp_3, freq_3, phase_3)
    etalon_4 = etalon(x_, amp_4, freq_4, phase_4)
    etalons = etalon_1 + etalon_2 + etalon_3 + etalon_4

    return etalons
def linear_model(params, x_, pressure_num, DF): 
    y_a = np.float(params['y_a_'+ str(pressure_num) ])
    y_b = np.float(params['y_b_'+ str(pressure_num)])
    y_c = np.float(params['y_b_' + str(pressure_num)])



    base =   y_a + y_b*(x_) + y_c*(x_*x_)
    return base



def sim_model(params, x, data, pressure_list,DF, x_min, fit_column, x_column):
    #Baseline Parameters

    MID = []
    per_O2 = np.float(params['per_O2'])
    concentration = {'O2' : per_O2 / 100}
    for isotope in isotopes: # Currently based on one molecule
        if natural_abundance == True:
            MID.append((ISO_ID[isotope][ISO_ID_INDEX['M']],ISO_ID[isotope][ISO_ID_INDEX['I']], concentration[ISO_ID[isotope][ISO_ID_INDEX['mol_name']]]* ISO_ID[isotope][ISO_ID_INDEX['abundance']]))
        elif natural_abundance == False:
            MID.append((ISO_ID[isotope][ISO_ID_INDEX['M']],ISO_ID[isotope][ISO_ID_INDEX['I']], concentration[ISO_ID[isotope][ISO_ID_INDEX['mol_name']]] ))#* ISO_ID[isotope][ISO_ID_INDEX['abundance']]

    
    # Line Parameters
    for i in range(0, length('tmp')): 
        if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DF[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DF[x_column].max() + wave_range):
            if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
                LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] = np.float(params['nu_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] = np.float(params['sw_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i] = np.float(params['gamma_air_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i] = np.float(params['delta_air_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i] = np.float(params['SD_air_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i] = np.float(params['nu_HT_air_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i] = np.float(params['y_air_%i'%(i)])  
                LOCAL_TABLE_CACHE['tmp']['data']['n_air'][i] = np.float(params['n_air_%i'%(i)])
                LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i] = np.float(params['deltap_air_%i'%(i)])
                #LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i] = np.float(params['n_He_%i'%(i)])
                #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i] = np.float(params['kappa_HT_air_%i'%(i)])            

    total_simulated = []
    
    total_residuals = []
    
    for pressure_num in pressure_list:
        for i in range(0, length('tmp')): 
            if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DF[x_column].min()) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DF[x_column].max() ):
                if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
                    LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] = np.float(params['y_factor_%i_%i'%(i, pressure_num)])*np.float(params['sw_%i'%(i)])
                     
                    
        pressure = DF[DF['Nominal Pressure'] == pressure_num]['Cavity Pressure /Torr'].mean()*0.00131579
        temperature = DF[DF['Nominal Pressure'] == pressure_num]['Cavity Temperature Side 2 /C'].mean() +273.15
        x_shift = np.float(params['x_shift_' + str(pressure_num)])
        

        nu_Drouin,Drouin_coef = ABSORB_COEFF_temp(Components = MID, 
                                        SourceTables='tmp',OmegaStep=0.005,Environment={'p':pressure,'T':temperature}, 
                                        GammaL = 'gamma_air', 
                                       IntensityThreshold = IntensityThreshold, HITRAN_units =False, 
                                           WavenumberWingHW = 150)
        Drouin_Alpha = Drouin_coef*1e6 
        freq_Drouin = nu_Drouin
        freq_Drouin += x_shift
        f = interpolate.interp1d(freq_Drouin, Drouin_Alpha, fill_value = 0, bounds_error = False)
        x_pressure = DF[(DF['Filter'] ==0) & (DF['Nominal Pressure'] == pressure_num)][x_column].values
        x_min_pressure =  DF[(DF['Filter'] ==0) & (DF['Nominal Pressure'] == pressure_num)][x_column].min()
        y = DF[(DF['Filter'] ==0) & (DF['Nominal Pressure'] == pressure_num)][fit_column].values
        

        Drouin_coef_interp_lm = f(x_pressure)
        etalons = etalon_model(params, x_pressure - np.float(x_min_pressure) ,pressure_num ,DF)
        baseline = linear_model(params, x_pressure - np.float(x_min_pressure) , pressure_num, DF)
        

        simulated_spectra = Drouin_coef_interp_lm +   baseline +etalons 
        residuals = simulated_spectra - y
        total_simulated = np.append(total_simulated, simulated_spectra)
        total_residuals = np.append(total_residuals, residuals)
 
    total_residuals = np.asarray(total_residuals)
    return total_residuals 






In [8]:
def summary_result_file(result, lineshape):
    background_values = []
    linelist_values = []
    fit_lines= []
    for k in (list(params.keys())):
        if 'sw_' in k:
            fit_lines = np.append(fit_lines,(str(k)[str(k).find('_') + 1:]))
        if 'per_O2' in k:
            background_values = np.append(background_values, k)
        elif 'x_shift' in k:
            background_values = np.append(background_values, k)
        elif ('y_a_' in k) or ('y_b_' in k) or ('y_c_' in k):
            background_values = np.append(background_values, k)
        elif ('amp' in k) or ('freq' in k) or ('phase' in k) or ('alpha' in k):
            background_values = np.append(background_values, k)
        else:
            linelist_values = np.append(linelist_values, k)

    BackgroundParameter_FitResults = pd.DataFrame()
    for bkg in background_values:

        fit_value = np.float((str(result.params[bkg]).split()[2][6:]))
        fit_param = bkg
        if (str(result.params[bkg]).split()[3]) == '+/-':
            fit_error = np.float((str(result.params[bkg]).split()[4][:-1]))
            low_bound = np.float( (str(result.params[bkg]).split()[5][8:str(result.params[bkg]).split()[5].find(':')]))
            upper_bound = np.float( (str(result.params[bkg]).split()[5][str(result.params[bkg]).split()[5].find(':') +1:str(result.params[bkg]).split()[5].find(']')]))
        elif (str(result.params[bkg]).split()[3]) == '(fixed),':
            fit_error = np.NaN
            low_bound = (str(result.params[bkg]).split()[4][8:str(result.params[bkg]).split()[4].find(':')])
            upper_bound = (str(result.params[bkg]).split()[4][str(result.params[bkg]).split()[4].find(':') + 1:str(result.params[bkg]).split()[4].find(']') ])
        d = pd.DataFrame({'Line': 'background','Parameter': fit_param, 
             'Fit Value' : fit_value, 'Fit Error' : fit_error, 
             'Lower Bound': low_bound, 'Upper Bound': upper_bound, 
                         'Lineshape': lineshape}, index = [0])
        BackgroundParameter_FitResults = BackgroundParameter_FitResults.append(d)  
    BackgroundParameter_FitResults = BackgroundParameter_FitResults[['Line', 'Lineshape', 'Parameter', 'Fit Value', 'Fit Error', 'Lower Bound', 'Upper Bound']]
    BackgroundParameter_FitResults = BackgroundParameter_FitResults.reset_index(drop = True)
    #print (BackgroundParameter_FitResults)

    # Line Parameter
    LineParameter_FitResults = pd.DataFrame()
    for fit_line in fit_lines:
        for ln in linelist_values:
            if str(ln).find(str(fit_line)) != -1: 
                fit_value = np.float((str(result.params[ln]).split()[2][6:]))
                fit_param = ln.replace('_' + fit_line, '')
                if (str(result.params[ln]).split()[3]) == '+/-':
                    fit_error = np.float((str(result.params[ln]).split()[4][:-1]))
                    low_bound = np.float( (str(result.params[ln]).split()[5][8:str(result.params[ln]).split()[5].find(':')]))
                    upper_bound = np.float( (str(result.params[ln]).split()[5][str(result.params[ln]).split()[5].find(':') +1:str(result.params[ln]).split()[5].find(']')]))
                elif (str(result.params[ln]).split()[3]) == '(fixed),':
                    fit_error = np.NaN
                    low_bound = (str(result.params[ln]).split()[4][8:str(result.params[ln]).split()[4].find(':')])
                    upper_bound = (str(result.params[ln]).split()[4][str(result.params[ln]).split()[4].find(':') + 1:str(result.params[ln]).split()[4].find(']') ])
                d = pd.DataFrame({'Line': fit_line,'Parameter': fit_param, 
                     'Fit Value' : fit_value, 'Fit Error' : fit_error, 
                     'Lower Bound': low_bound, 'Upper Bound': upper_bound, 
                                 'Lineshape': lineshape}, index = [0])
                LineParameter_FitResults = LineParameter_FitResults.append(d)
    LineParameter_FitResults = LineParameter_FitResults[['Line', 'Lineshape', 'Parameter', 'Fit Value', 'Fit Error', 'Lower Bound', 'Upper Bound']]
    LineParameter_FitResults = LineParameter_FitResults.reset_index(drop = True)
    #print (LineParameter_FitResults)    

    return (BackgroundParameter_FitResults, LineParameter_FitResults)

In [9]:
def diffusion_coefficient(T, D):
    k_B = 1.380648813*10**-16 # erg/K
    T = 296 # (K)
    m = molecularMass(7,1) / (6.0221409*10**23) # (g)
    D = 0.178 #(cm^2 / s)
    nuVC = (k_B*T / (m*D)) / (101325 * 10000000  * 0.29587215) # fudge factor of 10 in here that needs to be fixed.
    # returns in cm-1/atm
    #nuVC = 0.5
    return (nuVC)



In [ ]:
import time


os.chdir(hapi)

start_time = time.time()

# allowing the outside lines to float is key!  But you might want to do just the main lines and then let them float.  
# also don't float the aw term
params = Parameters()
###LINE SHAPE PARAMETERS
x_column = 'Wavenumber (FSR)'
fit_column = 'Alpha - Baseline Corrected'


lineshape = 'SDVP' #VP, NGP, NGP-LM, SDVP-LM, SDNGP, SDNGP-LM


DBfromsource = True
center_vary = False
intensity_vary = True
width_vary = False
shift_vary = False
sd_vary = False
nuVC_vary = False
y_vary = False

fix_temp_dep = False
width_temp_vary = False
shift_temp_vary = False
sd_temp_vary = False
nuVC_temp_vary = False

Filter_save = True

### BASELINE AND ETALONS

copy_etalon = True
lock_etalons =  False
etalon_1_on =  True
etalon_2_on = False
etalon_3_on = False
etalon_4_on = False

lock_baseline = False
baseline_order =1 #(0, 1, 2)
baseline_vary = True
xshift_vary = True
x_shift = -0.005


# Define Background and Etalon Parameters
params.add('per_O2', percent_oxygen, vary = True)
for pressure_num in pressure_list:
    params.add('x_shift_%i'%(pressure_num),-0.005, vary = xshift_vary , min = -0.05, max = 0.05) #-0.005230285
    
    #Baseline Parameters
    if baseline_order == 2:
        params.add('y_a_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_b_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
    if baseline_order == 1:
        params.add('y_a_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_b_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)
    if baseline_order==0:
        params.add('y_a_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_b_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)
    
    # Define Etalon Parameters
    if etalon_1_on == False:  
        params.add('amp_1_%i'%(pressure_num),0, vary=False)
        params.add('freq_1_%i'%(pressure_num), 1 / etalon_mirror[1], vary = False, min = (1 / etalon_mirror[1])*0.8, max = (1 / etalon_mirror[1])*1.2)
        params.add('phase_1_%i'%(pressure_num),0, vary = False)
        #params.add('alpha_1_%i'%(pressure_num), 1, vary = False)
    elif etalon_1_on == True:
        params.add('amp_1_%i'%(pressure_num),etalon_mirror[0], vary=True, min = 0.5*etalon_mirror[0], max = 2*etalon_mirror[0])
        params.add('freq_1_%i'%(pressure_num), 1 / etalon_mirror[1], vary = False, min = (1 / etalon_mirror[1])*0.8, max = (1 / etalon_mirror[1])*1.2)
        params.add('phase_1_%i'%(pressure_num),0, vary = True, min = -np.pi, max = np.pi)
        #params.add('alpha_1_%i'%(pressure_num), 1, vary = False, min = 0.95, max = 1.05)
    if etalon_2_on == False:  
        params.add('amp_2_%i'%(pressure_num),0, vary=False)
        params.add('freq_2_%i'%(pressure_num),1 / etalon_window[1], vary = False)
        params.add('phase_2_%i'%(pressure_num),0, vary =False)
        #params.add('alpha_2_%i'%(pressure_num), 1, vary = False)
    if etalon_2_on == True:
        params.add('amp_2_%i'%(pressure_num),etalon_window[0], vary=False, min = etalon_window[0]*0.5, max = etalon_window[0]*2)
        params.add('freq_2_%i'%(pressure_num),1 / etalon_window[1], vary = False, min = (1 /  etalon_window[1])*0.8, max = (1 / etalon_window[1])*1.2)
        params.add('phase_2_%i'%(pressure_num),0, vary =True, min = -np.pi, max = np.pi)
        #params.add('alpha_2_%i'%(pressure_num), 1, vary = False, min = 0.98, max = 1.02)
    if etalon_3_on == False:  
        params.add('amp_3_%i'%(pressure_num),0, vary=False)
        params.add('freq_3_%i'%(pressure_num),1 / etalon_slow[1], vary = False)
        params.add('phase_3_%i'%(pressure_num),0, vary =False)
        #params.add('alpha_3_%i'%(pressure_num), 1, vary = False)
    if etalon_3_on == True:
        params.add('amp_3_%i'%(pressure_num) ,etalon_slow[0], vary=True, min =  etalon_slow[0]*0.5, max = etalon_slow[0]*2)
        params.add('freq_3_%i'%(pressure_num),1 / etalon_slow[1], vary = False, min = (1 /  etalon_slow[1])*0.8, max = (1 / etalon_slow[1])*1.2)
        params.add('phase_3_%i'%(pressure_num),0, vary =True, min = -np.pi, max = np.pi)
        #params.add('alpha_3_%i'%(pressure_num), 1, vary = False, min = 0.90, max = 1.10)
    if etalon_4_on == False:  
        params.add('amp_4_%i'%(pressure_num),0, vary=False)
        params.add('freq_4_%i'%(pressure_num),1 / etalon_fast[1], vary = False)
        params.add('phase_4_%i'%(pressure_num),0, vary =False)
        #params.add('alpha_4_%i'%(pressure_num), 1, vary = False)
    if etalon_4_on == True:
        params.add('amp_4_%i'%(pressure_num),etalon_fast[0], vary=False, min = etalon_fast[0]*0.5, max = etalon_fast[0]*2)
        params.add('freq_4_%i'%(pressure_num),1 / etalon_fast[1], vary = False, min = (1 /  etalon_fast[1])*0.8, max = (1 /etalon_fast[1])*1.2)
        params.add('phase_4_%i'%(pressure_num),0, vary =True, min = -np.pi, max = np.pi)
        #params.add('alpha_4_%i'%(pressure_num), 1, vary = False, min = 0.98, max = 1.02)


# Define Lineshape parameters
if DBfromsource == True:
    Cond = ('AND',('BETWEEN','nu',np.floor(DATA[x_column].min() - 4*wave_range),np.ceil(DATA[x_column].max() + 4*wave_range)),('>=','sw',IntensityThreshold))
    select('Drouin_2016',Conditions=Cond,DestinationTableName='tmp')
    #select('HITRAN_O2',Conditions=Cond,DestinationTableName='tmp')
if fix_temp_dep == True:
    LOCAL_TABLE_CACHE['tmp']['data']['n_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['n_air'])*[1]
    LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'])*[0]
    #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'])*[1] #nuVC
    #LOCAL_TABLE_CACHE['tmp']['data']['n_He'] = len( LOCAL_TABLE_CACHE['tmp']['data']['n_He'])*[0] #aw temp
elif fix_temp_dep == False:
    # make so it sets to correct values
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['n_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val] = 1
        if LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][val] = 0
        #if LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][val] == 0:
            #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][val] = 1
        #if LOCAL_TABLE_CACHE['tmp']['data']['n_He'][val] == 0:
            #LOCAL_TABLE_CACHE['tmp']['data']['n_He'][val] = 0        

if lineshape == 'VP': #lineshape = 'VP' #VP, NGP, NGP-LM, SDVP-LM, SDNGP, SDNGP-LM
    sd_vary = False
    nuVC_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]
    
elif lineshape =='NGP':
    sd_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178)) 
    
elif lineshape =='NGP-LM':
    sd_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178)) 
elif lineshape =='SDVP':
    nuVC_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            a_w_ = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = a_w_#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
            
elif lineshape =='SDVP-LM':
    nuVC_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
elif lineshape =='SDNGP':
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] =(diffusion_coefficient(296, 0.178))
elif lineshape =='SDNGP-LM':
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178))
    

for i in range(0, length('tmp')):
    if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
        if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
            params.add('nu_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]), vary = center_vary, min = np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i])-0.25,  max = np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i])+0.25)
            params.add('sw_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['sw'][i]), vary =False)
            params.add('gamma_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]), vary = width_vary, min = 0.5*np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]), max =  2*np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]))
            params.add('n_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_air'][i]), vary = width_temp_vary, min = 0, max = 2)
            params.add('delta_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), vary = shift_vary, min = 0.5*np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), max =  2*np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]))
            params.add('deltap_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i]), vary = shift_temp_vary, min = -1, max = 1)           

            
            if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i] != 0:
                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = sd_vary, min = 0.1* np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), max = 10*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]))#, min = 0.5*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), max =  2*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]))
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = sd_temp_vary, min = -2, max = 2)
            else:
                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = False)
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = False)

            if LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i] != 0:
                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = y_vary, min = -0.1, max = 0.1)#, min = -10*np.abs(np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i])), max =  10*np.abs(np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i])))
            else:
                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = y_vary)
                
            if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i] != 0:
                params.add('nu_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), vary = nuVC_vary, min = 0.1*np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), max = 10*np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]) ) 
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = nuVC_temp_vary, min = -2, max = 2) 
            else:
                params.add('nu_HT_air_%i'%(i), 0, vary = False)
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = False)
            
            if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]> DATA[x_column].min()) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max()):
                for pressure_num in pressure_list:
                    params.add('y_factor_%i_%i'%(i, pressure_num),1, vary = intensity_vary, min = 0.5, max = 2) 
                    
                 
            if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].min()) or (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].max()):
                params.add('sw_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['sw'][i]), vary = intensity_vary)
                
                params.add('nu_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]), vary = False)
                
                params.add('delta_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), vary = False)
                params.add('deltap_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i]), vary = False)
                
                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = False)
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = False)
                
                params.add('nu_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), vary = False)
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = False)
                
                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = False)
                
if DBfromsource == False:
    for i in range(0, length('tmp')):
        if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
            if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
                if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]> DATA[x_column].min()) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max()):
                    for pressure_num in pressure_list:
                        print (pressure_num)
                        params.add('y_factor_%i_%i'%(i, pressure_num),np.float(str(result.params['y_factor_%i_%i'%(i, pressure_num)]).split()[2][6:]) , vary = intensity_vary, min = 0.5, max = 2) 
                        


if copy_etalon == True:
    for pressure_num in pressure_list:
                           
        params.add('y_a_%i'%(pressure_num), np.float(str(result.params['y_a_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary, min = -1, max = 1)
        if baseline_order >0:
            params.add('y_b_%i'%(pressure_num), np.float(str(result.params['y_b_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary, min = -1, max = 1)
        if baseline_order > 1:
            params.add('y_c_%i'%(pressure_num), np.float(str(result.params['y_c_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary, min = -1, max = 1)

        if etalon_1_on == True:
            if np.float(str(result.params['amp_1_%i'%(pressure_num)]).split()[2][6:]) != 0:
                params.add('amp_1_%i'%(pressure_num),np.float(str(result.params['amp_1_%i'%(pressure_num)]).split()[2][6:]), vary=True, min = 0.5*etalon_mirror[0], max = 2*etalon_mirror[0])
                params.add('phase_1_%i'%(pressure_num),np.float(str(result.params['phase_1_%i'%(pressure_num)]).split()[2][6:]), vary = True, min = -np.pi, max = np.pi)
                #params.add('alpha_1_%i'%(pressure_num),np.float(str(result.params['alpha_1_%i'%(pressure_num)]).split()[2][6:]), vary = False, min = 0.98, max =1.02)
        if etalon_2_on == True:
            if np.float(str(result.params['amp_2_%i'%(pressure_num)]).split()[2][6:]) != 0:
                params.add('amp_2_%i'%(pressure_num),np.float(str(result.params['amp_2_%i'%(pressure_num)]).split()[2][6:]), vary=True, min = etalon_window[0]*0.5, max = etalon_window[0]*2)
                params.add('phase_2_%i'%(pressure_num),np.float(str(result.params['phase_2_%i'%(pressure_num)]).split()[2][6:]), vary =True, min = -np.pi, max = np.pi)
                #params.add('alpha_2_%i'%(pressure_num),np.float(str(result.params['alpha_2_%i'%(pressure_num)]).split()[2][6:]), vary =False, min = 0.98, max = 1.02)
        if etalon_3_on == True: 
            if np.float(str(result.params['amp_3_%i'%(pressure_num)]).split()[2][6:]) != 0:           
                params.add('amp_3_%i'%(pressure_num),np.float(str(result.params['amp_3_%i'%(pressure_num)]).split()[2][6:]), vary=False, min =  etalon_slow[0]*0.5, max = etalon_slow[0]*2)
                params.add('phase_3_%i'%(pressure_num),np.float(str(result.params['phase_3_%i'%(pressure_num)]).split()[2][6:]), vary =True, min = -np.pi, max = np.pi)
                #params.add('alpha_3_%i'%(pressure_num),np.float(str(result.params['alpha_3_%i'%(pressure_num)]).split()[2][6:]), vary = False, min = 0.9, max =1.1)
        if etalon_4_on == True:
            if np.float(str(result.params['amp_4_%i'%(pressure_num)]).split()[2][6:]) != 0:
                params.add('amp_4_%i'%(pressure_num),np.float(str(result.params['amp_4_%i'%(pressure_num)]).split()[2][6:]), vary=False, min = etalon_fast[0]*0.5, max = etalon_fast[0]*2)
                params.add('phase_4_%i'%(pressure_num),np.float(str(result.params['phase_4_%i'%(pressure_num)]).split()[2][6:]), vary =True, min = -np.pi, max = np.pi)
                #params.add('alpha_4_%i'%(pressure_num),np.float(str(result.params['alpha_4_%i'%(pressure_num)]).split()[2][6:]), vary = False, min = 0.98, max =1.02)
for pressure_num in pressure_list:
    if lock_baseline == True:
        params.add('y_a_%i'%(pressure_num), np.float(str(result.params['y_a_%i'%(pressure_num)]).split()[2][6:]), vary =False)
        params.add('y_b_%i'%(pressure_num), np.float(str(result.params['y_b_%i'%(pressure_num)]).split()[2][6:]), vary =False)
        params.add('y_c_%i'%(pressure_num), np.float(str(result.params['y_c_%i'%(pressure_num)]).split()[2][6:]), vary =False)

    if lock_etalons == True:

        if etalon_1_on == True:
            params.add('amp_1%i'%(pressure_num),np.float(str(result.params['amp_1%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_1%i'%(pressure_num),np.float(str(result.params['phase_1%i'%(pressure_num)]).split()[2][6:]), vary = False)
            #params.add('alpha_1%i'%(pressure_num),np.float(str(result.params['alpha_1%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_2_on == True:
            params.add('amp_2%i'%(pressure_num),np.float(str(result.params['amp_2%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_2%i'%(pressure_num),np.float(str(result.params['phase_2%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_2%i'%(pressure_num),np.float(str(result.params['alpha_2%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_3_on == True: 
            params.add('amp_3%i'%(pressure_num),np.float(str(result.params['amp_3%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_3%i'%(pressure_num),np.float(str(result.params['phase_3%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_3%i'%(pressure_num),np.float(str(result.params['alpha_3%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_4_on == True:
            params.add('amp_4%i'%(pressure_num),np.float(str(result.params['amp_4%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_4%i'%(pressure_num),np.float(str(result.params['phase_4%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_4%i'%(pressure_num),np.float(str(result.params['alpha_4%i'%(pressure_num)]).split()[2][6:]), vary = False)


ignore_pts = []
filter_pts = []


for pt in ignore_pts:
    DATA.loc[(DATA.index == pt) , 'Filter'] = 1
for pt in filter_pts:
    DATA.loc[(DATA.index == pt)  , 'Filter'] = 2

x = DATA[(DATA['Filter'] ==0) ][x_column].values
x_min =  DATA[(DATA['Filter'] ==0) ][x_column].values.min()
y = DATA[(DATA['Filter'] ==0) ][fit_column].values
pressure_label =   DATA[(DATA['Filter'] ==0) ]['Nominal Pressure']  

                          

minner = Minimizer(sim_model, params,fcn_args=(x, y, pressure_list, DATA, x_min, fit_column, x_column), iter_cb = max_iter)
result = minner.minimize(method = 'leastsq')#'
final = y + result.residual
                           
fit_results = pd.DataFrame()
fit_results['Pressure'] = pressure_label
fit_results['Wavenumber'] = x
fit_results['Experiment'] = y
fit_results['Model'] = final
fit_results['Residuals'] = result.residual


bkg_result, param_result = summary_result_file(result, lineshape)
print (param_result)
#print (bkg_result)


# Update Lineshape parameters
for i in range(0, length('tmp')):
    if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
        if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
            LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] = np.float(str(result.params['nu_%i'%(i)]).split()[2][6:]) 
            LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] = np.float(str(result.params['sw_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i] = np.float(str(result.params['gamma_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i] = np.float(str(result.params['delta_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i] = np.float(str(result.params['SD_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i] = np.float(str(result.params['y_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i] = np.float(str(result.params['nu_HT_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['n_air'][i] = np.float(str(result.params['n_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i] = np.float(str(result.params['deltap_air_%i'%(i)]).split()[2][6:])
            #LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i] = np.float(str(result.params['n_He_%i'%(i)]).split()[2][6:])
            #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i] = np.float(str(result.params['kappa_HT_air_%i'%(i)]).split()[2][6:])
            
f = 0
g = len(pressure_list)
fig = plt.figure(figsize = (16,10))
gs = gridspec.GridSpec(2, len(pressure_list), height_ratios=[3, 1])
for pressure_num in pressure_list:

    
    P_segment = fit_results[fit_results['Pressure'] == pressure_num]
    QF = np.around((P_segment['Experiment'].max() - P_segment['Experiment'].min())  / P_segment['Residuals'].std(), 0)
     
    
    
    ax0 = plt.subplot(gs[f])
    ax0.plot(P_segment['Wavenumber'], P_segment['Model'], "-")
    ax0.plot(P_segment['Wavenumber'], P_segment['Experiment'], ".")
    ax0.text(0.25, 0.95,'QF: ' + str(QF), horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes)
    ax0.set_title(str(pressure_num) + ' Torr')
    ax0.plot()
        
    ax0.set_ylim(-0.05, 1.1*fit_results['Experiment'].max())
    #ax0.set_xlim(13090, 13110)

    ax1 = plt.subplot(gs[g])
    ax1.plot(P_segment['Wavenumber'],P_segment['Residuals'], ".")
   
        
    if f == 2:
        ax1.set_xlabel('Frequency (MHz)')
    if f ==0:
        ax0.set_ylabel('$\\frac{1}{c \\tau} (ppm/cm)$')
        ax1.set_ylabel('Residuals')
        ax0.set_xticks([])
    else:
        ax0.set_xticks([])
        ax0.set_yticks([])
        ax1.set_yticks([])
    #ax1.set_xlim(13090, 13110)
    ax1.set_ylim(-1.1*np.max(np.abs(fit_results['Residuals'].values)), 1.1*np.max(np.abs(fit_results['Residuals'].values)))
    f +=1
    g+=1
plt.tight_layout()


os.chdir(path)


#Saves fit results as a Dataframe
fit_results.to_csv(save_name + '_spectra.csv', index = False)
fit_param_result = param_result.append(bkg_result)
fit_param_result.to_csv(save_name + '_params.csv', index = False)



